# Necessary Packages

In [203]:
import PyPDF2 as pdf
import pandas as pd
import numpy as np
import math
import statistics
import operator
import matplotlib.pyplot as plt
import traitlets
from IPython.display import display
from ipywidgets import widgets
from tkinter import Tk, filedialog
from tokenize import tokenize
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import io
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import win32com.client
import codecs
from bs4 import BeautifulSoup
import textract
import docx
import os
import re
from tkinter import Tk, filedialog
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer

In [194]:
stop_words = frozenset(stopwords.words('english'))

# Pick File where syllabai are located 

In [150]:
root = Tk()
root.directory = filedialog.askdirectory()
directory = root.directory
root.withdraw() #hide the main window 

''

In [228]:
# initialize a list to add parsed text to
text_list = []

# Loop through the files in the directory
for filename in os.listdir(directory)[:500]: #only did 500 becuase it takes forever
    first_file = filename
    path = directory + '/' + first_file
    
    #handle parsing for different file types
    if '.pdf' in first_file:
        text = convert_pdf_to_txt_2(path)
    elif '.doc' in first_file:
        text = doc_parser(path)
    else:
        text = htm_parser(path)
    text_list.append(text)

KeyboardInterrupt: 

### Initial Processing of Data to create TF-IDF

In [207]:
cv=CountVectorizer(max_df=0.85,stop_words=stop_words)
word_count_vector=cv.fit_transform(text_list)

In [208]:
list(cv.vocabulary_.keys())[:10]

['african',
 'middle',
 'eastern',
 'languages',
 'literatures',
 'language',
 'center',
 'university',
 'amelang',
 'beginning']

In [209]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

### Testing the keyword generation for document 

In [212]:
feature_names=cv.get_feature_names()
number = np.random.randint(0, 500)

In [222]:
count = 0
test_list = []
for filename in os.listdir(directory):
    if count == number:
        test_file = filename
        path = directory + '/' + test_file
        if '.pdf' in test_file:
            text = convert_pdf_to_txt_2(path)
        elif '.doc' in test_file:
            text = doc_parser(path)
        else:
            text = htm_parser(path)
        test_list.append(text)
    count+= 1

In [227]:
feature_names=cv.get_feature_names()
 
# get the document that we want to extract keywords from
doc=test_list[0]
 
#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
 
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
 
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)
 
# now print the results
print("\n=====Doc=====")
print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])
 


=====Doc=====
 mark twain and the assault of laughter stanford university autumn english t mw hilton obenzinger office hours m w pm pm and by appointment office tba office phone tba home phone email obenzinger stanford edu for your human race in its poverty has unquestionably one really effective weapon laughter power money persuasion supplication persecution these can lift at a colossal humbug push it a little weaken it a little century by century but only laughter can blow it to rags and atoms at a blast against the assault of laughter nothing can stand from the mysterious stranger manuscripts description mark twain continues to delight and to disturb in this course we will examine how an outrageous phunny phellow notorious liar and irreverent blasphemer became a moral barometer the abraham lincoln of our literature against the assault of laughter nothing can stand twain wrote and his fictions satires and burlesques provided a constant comic barrage against the pretensions of his da

### Necessary Functions Defined below 

In [226]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [173]:
def convert_pdf_to_txt_2(path):
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)
        
    fp.close()
    device.close()
    text = retstr.getvalue()
    text_vec = text.split('\n')
    text_total =  ' '
    for word in text_vec:
        text_total += ' ' + word
    retstr.close()
    
     # lowercase
    text_total=text_total.lower()
    
    #remove tags
    text_total=re.sub("<!--?.*?-->","",text_total)
    
    # remove special characters and digits
    text_total=re.sub("(\\d|\\W)+"," ",text_total)
    
    return text_total

In [178]:
def htm_parser(path):
    f=codecs.open(path, 'r', encoding='latin-1')
    unsoup = f.read()
    soup = BeautifulSoup(unsoup)
    for script in soup(["script", "style"]):
        script.decompose()
    strips = list(soup.stripped_strings)
    text_total= ''
    for strip in strips:
        if (strip != 'Ê') and('Ê' not in strip):
            text_total += ' ' + strip.strip('\n')
     
    # lowercase
    text_total=text_total.lower()
    
    #remove tags
    text_total=re.sub("<!--?.*?-->","",text_total)
    
    # remove special characters and digits
    text_total=re.sub("(\\d|\\W)+"," ",text_total)
    
    return text_total

In [174]:
def doc_parser(path):
    doc = win32com.client.GetObject(path)
    text= doc.Range().Text.split('\r')
    text_total = ' '
    for word in text:
        if '' in word:
            word.replace('', '')
        if word != '':
            text_total += ' ' + word
            
     # lowercase
    text_total=text_total.lower()
    
    #remove tags
    text_total=re.sub("<!--?.*?-->","",text_total)
    
    # remove special characters and digits
    text_total=re.sub("(\\d|\\W)+"," ",text_total)
    
    return text_total